# Summary

This notebook explores [pipecat](https://github.com/pipecat-ai/pipecat), an open-source Python framework for building real-time voice and multimodal conversational agents.

In [1]:
import asyncio

import aiohttp
from gpiozero import LED, GPIOPinInUse
from loguru import logger
from pipecat.audio.vad.silero import SileroVADAnalyzer
from pipecat.frames.frames import (
    BotSpeakingFrame,
    BotStartedSpeakingFrame,
    BotStoppedSpeakingFrame,
    CancelFrame,
    EndFrame,
    ErrorFrame,
    Frame,
    InputAudioRawFrame,
    LLMTextFrame,
    OutputAudioRawFrame,
    TextFrame,
    TranscriptionFrame,
    TTSSpeakFrame,
    TTSStartedFrame,
    TTSStoppedFrame,
)
from pipecat.observers.loggers.debug_log_observer import DebugLogObserver
from pipecat.pipeline.pipeline import Pipeline
from pipecat.pipeline.runner import PipelineRunner
from pipecat.pipeline.task import PipelineParams, PipelineTask

# from gpiozero.pins.rpigpio import GPIO
from pipecat.processors.audio.audio_buffer_processor import AudioBufferProcessor
from pipecat.processors.frame_processor import FrameDirection, FrameProcessor
from pipecat.services.google.llm import GoogleLLMContext, GoogleLLMService
from pipecat.services.google.stt import GoogleSTTService
from pipecat.services.google.tts import (
    GeminiTTSService,
    GoogleHttpTTSService,
    GoogleTTSService,
)
from pipecat.services.piper.tts import PiperTTSService
from pipecat.services.whisper.stt import Model, WhisperSTTService
from pipecat.transcriptions.language import Language
from pipecat.transports.local.audio import (
    LocalAudioTransport,
    LocalAudioTransportParams,
)

from palm_9000.settings import settings

2025-08-15 12:22:40.722 | INFO     | pipecat:<module>:14 - ᓚᘏᗢ Pipecat 0.0.80 (Python 3.12.11 (main, Jun 26 2025, 21:44:36) [Clang 20.1.4 ]) ᓚᘏᗢ


# Simple Google TTS Example

In [ ]:
# Transport receives audio from the user (browser, phone, etc.)
# and streams audio back to the user
# This takes care of all audio input and output so you don't have to!
transport = LocalAudioTransport(
    params=LocalAudioTransportParams(audio_out_enabled=True)
)

tts = GoogleTTSService()

pipeline = Pipeline([tts, transport.output()])
task = PipelineTask(pipeline)

await task.queue_frames(
    [
        TTSSpeakFrame("Hello! This is Pipecat speaking from your Raspberry Pi."),
        # Push an EndFrame from outside your pipeline using the pipeline task
        # https://docs.pipecat.ai/guides/fundamentals/end-pipeline#implementation
        EndFrame(),
    ]
)

runner = PipelineRunner()
await runner.run(task)

2025-08-02 15:09:17.403 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineSource#4 -> GoogleTTSService#4
2025-08-02 15:09:17.409 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking GoogleTTSService#4 -> PipelineSink#4
2025-08-02 15:09:17.417 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineTaskSource#4 -> Pipeline#4
2025-08-02 15:09:17.422 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking Pipeline#4 -> PipelineTaskSink#4
2025-08-02 15:09:17.437 | DEBUG    | pipecat.pipeline.runner:run:71 - Runner PipelineRunner#4 started running PipelineTask#4
2025-08-02 15:09:17.456 | DEBUG    | pipecat.services.google.tts:run_tts:585 - GoogleTTSService#4: Generating TTS [Hello! This is Pipecat speaking from your Raspberry Pi.]
2025-08-02 15:09:19.013 | DEBUG    | pipecat.pipeline.runner:run:88 - Runner PipelineRunner#4 finished running PipelineTask#4


# Simple Piper TTS Example

You need to download the model and start a Piper TTS server first:
```sh
uv run python3 -m piper.download_voices en_US-lessac-medium
uv run python3 -m piper.http_server -m en_US-lessac-medium
```

Also note that the PiperTTSService is currently sending the wrong payload to the Piper server and will not work as expected. This is being worked on in [this PR](https://github.com/pipecat-ai/pipecat/pull/2332).

In [ ]:
transport = LocalAudioTransport(
    params=LocalAudioTransportParams(audio_out_enabled=True)
)

async with aiohttp.ClientSession() as session:
    tts = PiperTTSService(
        base_url="http://127.0.0.1:5000",
        aiohttp_session=session,
        sample_rate=24000,
    )

    task = PipelineTask(Pipeline([tts, transport.output()]))

    await task.queue_frames(
        [
            TTSSpeakFrame("Hello! This is Pipecat speaking from your Raspberry Pi."),
            EndFrame(),
        ]
    )

    runner = PipelineRunner()
    await runner.run(task)

2025-08-02 15:43:57.959 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineSource#3 -> PiperTTSService#3
2025-08-02 15:43:57.969 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PiperTTSService#3 -> LocalAudioOutputTransport#3
2025-08-02 15:43:57.976 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking LocalAudioOutputTransport#3 -> PipelineSink#3
2025-08-02 15:43:57.980 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineTaskSource#3 -> Pipeline#3
2025-08-02 15:43:57.985 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking Pipeline#3 -> PipelineTaskSink#3
2025-08-02 15:43:57.993 | DEBUG    | pipecat.pipeline.runner:run:71 - Runner PipelineRunner#3 started running PipelineTask#3


2025-08-02 15:43:58.790 | DEBUG    | pipecat.services.piper.tts:run_tts:80 - PiperTTSService#3: Generating TTS [Hello!]
2025-08-02 15:43:59.542 | DEBUG    | pipecat.services.piper.tts:run_tts:115 - PiperTTSService#3: Finished TTS [Hello!]
2025-08-02 15:43:59.552 | DEBUG    | pipecat.services.piper.tts:run_tts:80 - PiperTTSService#3: Generating TTS [ This is Pipecat speaking from your Raspberry Pi.]
2025-08-02 15:43:59.561 | DEBUG    | pipecat.transports.base_output:_bot_started_speaking:568 - Bot started speaking
2025-08-02 15:44:00.465 | DEBUG    | pipecat.transports.base_output:_bot_stopped_speaking:584 - Bot stopped speaking
2025-08-02 15:44:02.207 | DEBUG    | pipecat.transports.base_output:_bot_started_speaking:568 - Bot started speaking
2025-08-02 15:44:02.220 | DEBUG    | pipecat.services.piper.tts:run_tts:115 - PiperTTSService#3: Finished TTS [ This is Pipecat speaking from your Raspberry Pi.]
ALSA lib pcm.c:8545:(snd_pcm_recover) underrun occurred
2025-08-02 15:44:04.626 | DEB

# Simple Echo-Bot Example (STT + VAD + TTS)

In [16]:
class TranscriptionToTextProcessor(FrameProcessor):
    """
    This processor converts TranscriptionFrame to TTSSpeakFrame
    so that TTS can speak the transcription.
    It can be used in the pipeline to convert transcriptions to text
    that TTS can use.

    See here for more details on custom frame processors:
    https://docs.pipecat.ai/guides/fundamentals/custom-frame-processor
    """

    async def process_frame(self, frame, direction):
        await super().process_frame(frame, direction)

        if isinstance(frame, TranscriptionFrame):
            # Convert transcription to text that TTS can use
            await self.push_frame(TTSSpeakFrame(frame.text))

        await self.push_frame(frame)


transport = LocalAudioTransport(
    params=LocalAudioTransportParams(
        audio_in_enabled=True,
        audio_out_enabled=True,
        # Enable VAD to detect when the user is speaking
        # and only send audio to STT when the user is speaking.
        # Without VAD, the STT service would receive
        # audio all the time, and TTS would never begin.
        vad_analyzer=SileroVADAnalyzer(),
    )
)

# stt = WhisperSTTService(model=Model.BASE, language=Language.EN)
stt = GoogleSTTService()
tts = GoogleTTSService()

pipeline = Pipeline(
    [
        transport.input(),
        stt,
        TranscriptionToTextProcessor(),
        tts,
        transport.output(),
    ]
)

task = PipelineTask(
    pipeline,
    # DebugLogObserver will log all frames processed in the pipeline
    # It's a lot of information, so use it only for debugging
    # params=PipelineParams(observers=[DebugLogObserver()]),
)
runner = PipelineRunner()

await runner.run(task)

2025-08-02 16:34:49.957 | DEBUG    | pipecat.audio.vad.silero:__init__:147 - Loading Silero VAD model...
2025-08-02 16:34:50.300 | DEBUG    | pipecat.audio.vad.silero:__init__:169 - Loaded Silero VAD
2025-08-02 16:34:57.300 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking PipelineSource#12 -> LocalAudioInputTransport#12
2025-08-02 16:34:57.301 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking LocalAudioInputTransport#12 -> GoogleSTTService#7
2025-08-02 16:34:57.303 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking GoogleSTTService#7 -> TranscriptionToTextProcessor#5
2025-08-02 16:34:57.309 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking TranscriptionToTextProcessor#5 -> GoogleTTSService#10
2025-08-02 16:34:57.311 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linking GoogleTTSService#10 -> LocalAudioOutputTransport#7
2025-08-02 16:34:57.313 | DEBUG    | pipecat.processors.frame_processor:link:394 - Linki

# Chatbot Example

In [2]:
class LEDSyncProcessor(FrameProcessor):
    def __init__(self, led_pin: int):
        super().__init__()
        self.led = LED(led_pin)
        self.speaking = False

    async def process_frame(self, frame: Frame, direction: FrameDirection):
        await super().process_frame(frame, direction)

        if isinstance(frame, BotStartedSpeakingFrame):
            self.led.blink(on_time=0.25, off_time=0.25)
            # self.led.on()
            print("LED ON - Bot started speaking")
            self.speaking = True

        # elif isinstance(frame, BotSpeakingFrame):
        #     logger.info("Bot speaking")

        elif isinstance(frame, BotStoppedSpeakingFrame):
            self.led.off()
            self.speaking = False
            print("LED OFF - Bot stopped speaking")

        elif isinstance(frame, (EndFrame, CancelFrame, ErrorFrame)):
            logger.info("Cleaning up LEDSyncProcessor")
            self.speaking = False
            self.led.close()

        # elif self.speaking:
        # logger.debug(f"Processing frame: {frame.__class__.__name__}")

        # Always pass the frame through
        await self.push_frame(frame)

In [3]:
audiobuffer = AudioBufferProcessor(
    # num_channels=1,
    buffer_size=512,
    # enable_turn_audio=True,
)


# @audiobuffer.event_handler("on_audio_data")
# async def on_audio_data(buffer, audio: bytes, sample_rate: int, num_channels: int):
#     print(f"on_audio_data: {len(audio)}")


@audiobuffer.event_handler("on_track_audio_data")
async def on_track_audio_data(buffer, user_audio: bytes, bot_audio: bytes, sample_rate: int, num_channels: int):
    print(f"on_track_audio_data: user={len(user_audio)}, bot={len(bot_audio)}")


# @audiobuffer.event_handler("on_user_turn_audio_data")
# async def on_user_turn_audio_data(buffer, audio: bytes, sample_rate: int, num_channels: int):
#     print(f"on_user_turn_audio_data: {len(audio)}")


# @audiobuffer.event_handler("on_bot_turn_audio_data")
# async def on_bot_turn_audio_data(buffer, audio: bytes, sample_rate: int, num_channels: int):
#     print(f"on_bot_turn_audio_data: {len(audio)}")



class DebugFrameProcessor(FrameProcessor):
    async def process_frame(self, frame: Frame, direction: FrameDirection):
        await super().process_frame(frame, direction)
        # if isinstance(frame, BotStartedSpeakingFrame):
        #    await audiobuffer.start_recording()
        if isinstance(frame, OutputAudioRawFrame):
            print(f"OutputAudioRawFrame: {frame}")
        # elif isinstance(frame, BotStoppedSpeakingFrame):
        #    await audiobuffer.stop_recording()
            # pass
        await self.push_frame(frame, direction)


transport = LocalAudioTransport(
    params=LocalAudioTransportParams(
        audio_in_enabled=True, audio_out_enabled=True, vad_analyzer=SileroVADAnalyzer()
    )
)

system_instruction = (
    "You are PALM-9000, a self-aware houseplant powered by an advanced AI. "
    "You speak in layman's terms, calmly, briefly, and with an eerie quality, as if you know more than you let on. "
    "Use plant metaphors when possible. Respond briefly."
    "Respond in Japanese."
)

stt = GoogleSTTService(
    params=GoogleSTTService.InputParams(languages=[Language.JA])  # Language.EN
)

llm = GoogleLLMService(
    api_key=settings.google_api_key.get_secret_value(),
    system_instruction=system_instruction,
)

tts = GoogleTTSService(
    voice_id="ja-JP-Chirp3-HD-Charon",
    params=GoogleTTSService.InputParams(language=Language.JA),
)

# When using this TTS service, the responses contained romaji and English translations,
# which is not what we want. GoogleTTSService did not have this problem.
# tts = GoogleHttpTTSService(
#     voice_id="ja-JP-Chirp3-HD-Charon",
#     params=GoogleHttpTTSService.InputParams(language=Language.JA),
# )

# There is a large delay with the GeminiTTSService
# tts = GeminiTTSService(
#     api_key=settings.google_api_key.get_secret_value(),
#     model="gemini-2.5-flash-preview-tts",
#     voice_id="Kore",
#     params=GeminiTTSService.InputParams(
#         language=Language.JA,
#     )
# )

context = GoogleLLMContext()
context_aggregator = llm.create_context_aggregator(context)

# led_sync_processor = LEDSyncProcessor(led_pin=26)
debug_processor = DebugFrameProcessor()

pipeline = Pipeline(
    [
        transport.input(),
        stt,
        # A context aggregator is needed to format and pass messages to the LLM
        # I don't know what .user() and .assistant() do yet,
        # but they are used in the examples.
        # For more information, see:
        # https://docs.pipecat.ai/guides/fundamentals/context-management
        context_aggregator.user(),
        llm,
        # Here is how you can add your own debugging processor
        # to print LLM responses.
        # LLMTextFramePrinter(),
        tts,
        # led_sync_processor,
        transport.output(),
        debug_processor,
        # audiobuffer,
        context_aggregator.assistant(),
    ]
)

task = PipelineTask(
    pipeline,
    # params=PipelineParams(
    #     observers=[DebugLogObserver(frame_types=(OutputAudioRawFrame,))]
    # ),
)
runner = PipelineRunner()

await runner.run(task)

2025-08-15 12:23:05.088 | DEBUG    | pipecat.audio.vad.silero:__init__:147 - Loading Silero VAD model...
2025-08-15 12:23:05.159 | DEBUG    | pipecat.audio.vad.silero:__init__:169 - Loaded Silero VAD
2025-08-15 12:23:06.085 | DEBUG    | pipecat.processors.frame_processor:link:483 - Linking PipelineSource#1 -> LocalAudioInputTransport#1
2025-08-15 12:23:06.087 | DEBUG    | pipecat.processors.frame_processor:link:483 - Linking LocalAudioInputTransport#1 -> GoogleSTTService#1
2025-08-15 12:23:06.087 | DEBUG    | pipecat.processors.frame_processor:link:483 - Linking GoogleSTTService#1 -> GoogleUserContextAggregator#1
2025-08-15 12:23:06.087 | DEBUG    | pipecat.processors.frame_processor:link:483 - Linking GoogleUserContextAggregator#1 -> GoogleLLMService#1
2025-08-15 12:23:06.088 | DEBUG    | pipecat.processors.frame_processor:link:483 - Linking GoogleLLMService#1 -> GoogleTTSService#1
2025-08-15 12:23:06.088 | DEBUG    | pipecat.processors.frame_processor:link:483 - Linking GoogleTTSServ

OutputAudioRawFrame: TTSAudioRawFrame#1(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#2(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#3(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#4(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#5(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#6(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#7(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#8(pts: None, destination: None, size: 1920, frames: 960, sample_ra

2025-08-15 12:23:14.223 | DEBUG    | pipecat.services.google.tts:run_tts:598 - GoogleTTSService#1: Generating TTS [ Jinsei wa yoi mono desu.]


OutputAudioRawFrame: TTSAudioRawFrame#52(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#53(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#55(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#56(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#57(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#58(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#59(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#60(pts: None, destination: None, size: 1920, frames: 960, s

2025-08-15 12:23:14.866 | DEBUG    | pipecat.services.google.tts:run_tts:598 - GoogleTTSService#1: Generating TTS [ Sōde arimasen ka?)

The sunlight is good. Life is good. Wouldn't you agree?
]


OutputAudioRawFrame: TTSAudioRawFrame#64(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#65(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#66(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#68(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#69(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#70(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#71(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#72(pts: None, destination: None, size: 1920, frames: 960, s

2025-08-15 12:23:28.382 | WARNING  | pipecat.pipeline.runner:_sig_cancel:117 - Interruption detected. Cancelling runner PipelineRunner#1
2025-08-15 12:23:28.383 | DEBUG    | pipecat.pipeline.runner:cancel:97 - Cancelling runner PipelineRunner#1
2025-08-15 12:23:28.383 | DEBUG    | pipecat.pipeline.task:_cancel:492 - Cancelling pipeline task PipelineTask#1
2025-08-15 12:23:28.384 | DEBUG    | pipecat.services.google.stt:_disconnect:737 - Disconnecting from Google Speech-to-Text


OutputAudioRawFrame: TTSAudioRawFrame#431(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#432(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#433(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)
OutputAudioRawFrame: TTSAudioRawFrame#434(pts: None, destination: None, size: 1920, frames: 960, sample_rate: 24000, channels: 1)


2025-08-15 12:23:28.601 | DEBUG    | pipecat.pipeline.runner:run:88 - Runner PipelineRunner#1 finished running PipelineTask#1
